In [4]:
import numpy as np
import pylab as pl
import scipy as sp
import json
import csv
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

# Input your own file path to https://github.com/vanferdi/resource-rational-prediction/blob/main/Data/experiment.csv
file_path = 'experiment.csv'

Inputs = []
Guesses = []
ParticipantIDs = []
Input_Types = []
Hidden_states = []
Accuracy = []
Bonus_round = []

with open(file_path) as csvfile:
		spamreader = csv.reader(csvfile)
		for row in spamreader:
			input_ = row[7]
			guess_ = row[6]
			ID = row[0]
			hmm_states = row[13]
			accuracy = row[22]
			bonus_round = row[29]
			Inputs.append(input_)
			Guesses.append(guess_)
			ParticipantIDs.append(ID)
			Input_Types.append(row[15])
			Hidden_states.append(hmm_states)
			Accuracy.append(accuracy)
			Bonus_round.append(bonus_round)
		Inputs.pop(0)
		Guesses.pop(0)
		ParticipantIDs.pop(0)
		Input_Types.pop(0)
		Hidden_states.pop(0)
		Accuracy.pop(0)
		Bonus_round.pop(0)

with open(file_path) as csvfile:
		spamreader = csv.reader(csvfile)
		for row in spamreader:
			input_ = row[7]
			guess_ = row[6]
			ID = row[0]
			hmm_states = row[13]
			accuracy = row[22]
			bonus_round = row[29]
			Inputs.append(input_)
			Guesses.append(guess_)
			ParticipantIDs.append(ID)
			Input_Types.append(row[15])
			Hidden_states.append(hmm_states)
			Accuracy.append(accuracy)
			Bonus_round.append(bonus_round)
		Inputs.pop(0)
		Guesses.pop(0)
		ParticipantIDs.pop(0)
		Input_Types.pop(0)
		Hidden_states.pop(0)
		Accuracy.pop(0)
		Bonus_round.pop(0)

In [4]:
Input_Types[1]

'NoisyPeriodic'

In [5]:
len(Inputs[1])

445

In [5]:
def rate(hidden_states,guesses):
	# get statistics
	# this only works when things are well-sampled
	joint_xy = {}
	marginal_x = {}
	marginal_y = {}
	for i in range(len(hidden_states)):
		xy = str(hidden_states[i])+str(guesses[i])
		x = str(hidden_states[i])
		y = str(guesses[i])
		if xy in joint_xy:
			joint_xy[xy] = joint_xy[xy]+1
		else:
			joint_xy[xy] = 1
		if x in marginal_x:
			marginal_x[x] = marginal_x[x]+1
		else:
			marginal_x[x] = 1
		if y in marginal_y:
			marginal_y[y] = marginal_y[y]+1
		else:
			marginal_y[y] = 1
	# hack to add appropriate pseudocounts
	all_cs = np.unique(hidden_states)
	all_output = [0,1]
	if len(marginal_y)<2:
		marginal_y[str(2)] = 0
	for cs in all_cs:
		for output in all_output:
			xy = str(cs)+str(output)
			if xy in joint_xy:
				pass
			else:
				joint_xy[xy] = 0
	# find tot
	tot_xy = np.sum(list(joint_xy.values()))
	tot_x = np.sum(list(marginal_x.values()))
	tot_y = np.sum(list(marginal_y.values()))
	# get the entropies
	p_xy = np.asarray(list(joint_xy.values()))/tot_xy
	H_xy = -np.nansum(p_xy*np.log2(p_xy))
	p_x = np.asarray(list(marginal_x.values()))/tot_x
	H_x = -np.nansum(p_x*np.log2(p_x))
	p_y = np.asarray(list(marginal_y.values()))/tot_y
	H_y = -np.nansum(p_y*np.log2(p_y))
	# get the mutual information
	I_xy = H_x+H_y-H_xy
	return I_xy

In [6]:
def ngram_learner(n,data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	data_input1 = []
	hidden_states1 = []
	guesses1 = []
	guesses2 = []
	for i in range(n+1,len(data_input)):
		# figure out what the probabilities are of guessing
		# last n symbols
		foo = data_input[i-n-1:i-1]
		foo0 = foo+'0'
		foo1 = foo+'1'
		p0 = data_input[:i].count(foo0)+1
		p1 = data_input[:i].count(foo1)+1
		guess1 = np.argmax([p0,p1])
		guess2 = np.random.choice(2,size=1,p=[p0/(p0+p1),p1/(p0+p1)])
		guesses1.append(guess1)
		guesses2.append(guess2)
		hidden_states1.append(hidden_states[i-1])
		data_input1.append(data_input[i])
	# get rates and accuracies
	M = len(guesses1); guesses1 = guesses1[int(M/2):M]
	guesses2 = guesses2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses1)
	Rate2 = rate(hidden_states1,guesses2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([int(data_input[j])==guesses1[j] for j in range(int(M/2))])
	Acc2 = np.mean([int(data_input[j])==guesses2[j] for j in range(int(M/2))]); Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [8]:
print(Input_Types[0])
print(bad_learners(Inputs[0]))
for i in range(10):
  print(ngram_learner(i,Inputs[0],Hidden_states[0]))

Clumpy
([0, 0], [0.5, np.float64(0.6140888208269525)])
([np.float64(0.0), np.float64(0.0005106406115982232)], [np.float64(0.6134969325153374), np.float64(0.5122699386503068)])
([np.float64(0.9583015836671662), np.float64(0.0011640633039640136)], [np.float64(0.48307692307692307), np.float64(0.48923076923076925)])
([np.float64(0.7030778604454733), np.float64(0.32273922372254216)], [np.float64(0.6061538461538462), np.float64(0.5230769230769231)])
([np.float64(0.38164381118861623), np.float64(0.34458982847137465)], [np.float64(0.5432098765432098), np.float64(0.5493827160493827)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.3928748672364626), np.float64(0.35798311565589214)], [np.float64(0.5246913580246914), np.float64(0.5061728395061729)])
([np.float64(0.42322104748374123), np.float64(0.2871312101579009)], [np.float64(0.5541795665634675), np.float64(0.5232198142414861)])
([np.float64(0.43563357427977767), np.float64(0.20114142466163676)], [np.float64(0.5108359133126935), np.float64(0.4953560371517028)])
([np.float64(0.4724886632217), np.float64(0.2697966525608573)], [np.float64(0.5341614906832298), np.float64(0.5031055900621118)])
([np.float64(0.48385048497789573), np.float64(0.194804056048127)], [np.float64(0.5652173913043478), np.float64(0.4658385093167702)])
([np.float64(0.45741426460156775), np.float64(0.208528592688344)], [np.float64(0.5327102803738317), np.float64(0.5109034267912772)])


In [9]:
print(Input_Types[1])
print(bad_learners(Inputs[1]))
for i in range(10):
  print(ngram_learner(i,Inputs[1],Hidden_states[1]))

NoisyPeriodic
([0, 0], [0.5, np.float64(0.5460674157303371)])
([np.float64(0.0), np.float64(0.0029047301356441846)], [np.float64(0.5585585585585585), np.float64(0.536036036036036)])
([np.float64(0.8273863324079629), np.float64(0.5991124243675847)], [np.float64(0.9095022624434389), np.float64(0.8778280542986425)])
([np.float64(0.9999852306467514), np.float64(0.41613321263852354)], [np.float64(0.058823529411764705), np.float64(0.18552036199095023)])
([np.float64(0.9999852306467514), np.float64(0.5629593432025628)], [np.float64(0.05909090909090909), np.float64(0.16363636363636364)])
([np.float64(1.0), np.float64(0.5158875900539412)], [np.float64(0.9409090909090909), np.float64(0.8681818181818182)])
([np.float64(1.0), np.float64(0.5495033360017614)], [np.float64(0.9406392694063926), np.float64(0.8447488584474886)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.9999849596541528), np.float64(0.49852342885513456)], [np.float64(0.0593607305936073), np.float64(0.1689497716894977)])
([np.float64(0.9999849596541528), np.float64(0.39830697108062507)], [np.float64(0.05963302752293578), np.float64(0.22018348623853212)])
([np.float64(1.0), np.float64(0.4399341370246803)], [np.float64(0.9403669724770642), np.float64(0.8073394495412844)])
([np.float64(1.0), np.float64(0.4497787785866709)], [np.float64(0.9400921658986175), np.float64(0.8294930875576036)])


In [10]:
print(Input_Types[2])
print(bad_learners(Inputs[2]))
for i in range(10):
  print(ngram_learner(i,Inputs[2],Hidden_states[2]))

EvenProcess
([0, 0], [0.5, np.float64(0.5381062355658199)])
([np.float64(0.0), np.float64(0.0005407533039567536)], [np.float64(0.5324074074074074), np.float64(0.5416666666666666)])
([np.float64(0.3187791725115614), np.float64(0.02008922396406354)], [np.float64(0.4604651162790698), np.float64(0.5209302325581395)])
([np.float64(0.08592381874622945), np.float64(0.06289202331325083)], [np.float64(0.4790697674418605), np.float64(0.4558139534883721)])


/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: divide by zero encountered in log2
  H_y = -np.nansum(p_y*np.log2(p_y))
/tmp/ipython-input-2166640528.py:45: RuntimeWarning: invalid value encountered in multiply
  H_y = -np.nansum(p_y*np.log2(p_y))


([np.float64(0.2140230858094503), np.float64(0.11996995287528156)], [np.float64(0.48598130841121495), np.float64(0.5280373831775701)])
([np.float64(0.2237295050598307), np.float64(0.20775529133526005)], [np.float64(0.5046728971962616), np.float64(0.5)])
([np.float64(0.3812150748820715), np.float64(0.16892502800976494)], [np.float64(0.5023474178403756), np.float64(0.4788732394366197)])
([np.float64(0.35782965580958814), np.float64(0.14824164454846644)], [np.float64(0.5211267605633803), np.float64(0.5070422535211268)])
([np.float64(0.3941190697293615), np.float64(0.09888055492851322)], [np.float64(0.5141509433962265), np.float64(0.49056603773584906)])
([np.float64(0.43976514570880054), np.float64(0.06168527342530106)], [np.float64(0.5424528301886793), np.float64(0.46226415094339623)])
([np.float64(0.4616419095283124), np.float64(0.06267156193226264)], [np.float64(0.5497630331753555), np.float64(0.5639810426540285)])


In [39]:
def LSTM_learner(data_input,hidden_states):
	# first rate and accuracy is with no probability matching
	# second rate and accuracy is with probability matching
	timesteps = 5
	data_input1 = []
	hidden_states1 = []
	guesses_1 = [] # with no probability matching
	guesses_2 = [] # with probability matching
	for t in range(timesteps+1,len(data_input)):
		tf.keras.backend.clear_session()
		# train an LSTM to predict in MSE sense
		batch_x = []; batch_y = []
		for i in range(timesteps,t):
			batch_x.append([int(data_input[j]) for j in range(i-timesteps,i)])
			batch_y.append([int(data_input[i])])
		batch_x = np.asarray(batch_x)
		batch_y = np.asarray(batch_y)
		batch_x = batch_x.reshape((len(batch_x),timesteps,1))
		batch_y = batch_y.reshape((len(batch_y),1))
		batch_x = tf.convert_to_tensor(batch_x)
		batch_y = tf.convert_to_tensor(batch_y)
		#
		X = tf.keras.Input(name='X', shape=[timesteps, 1], dtype=tf.dtypes.float32)
		#
		lstm_output = tf.keras.layers.LSTM(50)(X)
		prediction = tf.keras.layers.Dense(1,activation='sigmoid')(lstm_output)
		opt = tf.keras.optimizers.Adam(learning_rate=5e-6)
		model = keras.Model(inputs=X, outputs=prediction)
		model.compile(optimizer=opt, loss='binary_crossentropy')
		model.fit(x=batch_x, y=batch_y, epochs=1000, verbose=0)
		xpreds = model.predict(batch_x[-1])
		data_input1.append(batch_y.numpy()[-1]); #print(batch_y.numpy()[-1])
		hidden_states1.append(hidden_states[t-1])
		guesses_1.append(np.round(xpreds[0][0])); p = np.array([1 - float(xpreds[0][0]), float(xpreds[0][0])], dtype=np.float64)
		guesses_2.append(np.random.choice(a=2,size=1,p=p/np.sum(p)))
	# get rates and accuracies
	M = len(guesses_1); guesses_1 = guesses_1[int(M/2):M]
	guesses_2 = guesses_2[int(M/2):M]
	data_input1 = data_input1[int(M/2):M]; hidden_states1 = hidden_states1[int(M/2):M]
	Rate1 = rate(hidden_states1,guesses_1)
	Rate2 = rate(hidden_states1,guesses_2)
	Rates = [Rate1,Rate2]
	Acc1 = np.mean([int(data_input[j])==guesses_1[j] for j in range(len(guesses_1))])
	Acc2 = np.mean([int(data_input[j])==guesses_2[j] for j in range(len(guesses_2))])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [7]:
def bad_learners(data_input):
	foo = [int(i) for i in data_input]
	freq1 = np.mean(foo); freq0 = 1-freq1 # best guess
	Rate1 = 0
	Rate2 = 0
	Rates = [Rate1,Rate2]
	Acc1 = 0.5
	Acc2 = np.max([freq0,freq1])
	Accuracies = [Acc1,Acc2]
	return Rates, Accuracies

In [40]:
r, a = LSTM_learner(Inputs[0][:100],Hidden_states[0][:100])
print(r)
print(a)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

/tmp/ipython-input-2166640528.py:41: RuntimeWarning: divide by zero encountered in log2
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
/tmp/ipython-input-2166640528.py:41: RuntimeWarning: invalid value encountered in multiply
  H_xy = -np.nansum(p_xy*np.log2(p_xy))
